<a href="https://colab.research.google.com/github/paella55/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/AG3_Jaime_Diaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AG3 - Actividad Guiada 3

### Jaime Díaz González

URL: https://github.com/paella55/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/AG3_Jaime_Diaz.ipynb

### Problema agente viajero

In [1]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7fca6befd828>)

In [3]:
!pip install tsplib95

    100% |████████████████████████████████| 1.6MB 16.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import tsplib95
import random
from math import e

problem = tsplib95.load_problem(file)


#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges()) 

In [0]:
#Devuelve el factorial de un numero
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
      
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion
                 
#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.wfunc(a,b)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [8]:
def busqueda_aleatoria(problem, N):
  
  Nodos = list(problem.get_nodes())
  
  distancia_final = 10000000
  solucion_final = -1
  
  for i in range(N):
    solucion_actual = crear_solucion(Nodos)    
    distancia_actual = distancia_total(solucion_actual, problem)
    
    if distancia_actual < distancia_final:
      solucion_final = solucion_actual
      distancia_final = distancia_actual
      
  print("Nuestra solución es: " + str(solucion_final))
  print("Nuestra distancia total es: " + str(distancia_final))
  return solucion_final

busqueda_aleatoria(problem, 5000)

Nuestra solución es: [0, 14, 13, 20, 33, 41, 12, 7, 36, 1, 11, 39, 24, 40, 16, 25, 2, 19, 3, 28, 10, 4, 15, 17, 21, 38, 23, 9, 34, 30, 8, 22, 29, 27, 18, 35, 31, 26, 5, 6, 32, 37]
Nuestra distancia total es: 3882


[0,
 14,
 13,
 20,
 33,
 41,
 12,
 7,
 36,
 1,
 11,
 39,
 24,
 40,
 16,
 25,
 2,
 19,
 3,
 28,
 10,
 4,
 15,
 17,
 21,
 38,
 23,
 9,
 34,
 30,
 8,
 22,
 29,
 27,
 18,
 35,
 31,
 26,
 5,
 6,
 32,
 37]

#### Búsqueda local

In [9]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion
                      
solucion = crear_solucion(Nodos)
print(solucion)
nueva_solucion = genera_vecina(solucion)
print(nueva_solucion)

[0, 41, 12, 2, 31, 20, 36, 35, 24, 3, 21, 37, 22, 32, 1, 19, 15, 16, 17, 9, 10, 4, 11, 26, 39, 5, 33, 14, 6, 34, 18, 30, 13, 7, 40, 28, 38, 23, 8, 27, 29, 25]
[0, 41, 12, 2, 31, 20, 36, 35, 37, 3, 21, 24, 22, 32, 1, 19, 15, 16, 17, 9, 10, 4, 11, 26, 39, 5, 33, 14, 6, 34, 18, 30, 13, 7, 40, 28, 38, 23, 8, 27, 29, 25]


In [38]:
def deberiamos_parar(distancias_anteriores):
  
  distancias_ultimas_mejores = 0
  for i in range(len(distancias_anteriores)-1):
    # Vamos recogiendo las distancias de una a la siguiente
    distancias_ultimas_mejores = distancias_ultimas_mejores + abs(distancias_anteriores[i]-distancias_anteriores[+1])
  
  # Si encontramos que la media de las diferencias es menor de 20, 
  # entonces debemos parar porque estamos ante un máximo local
  return distancias_ultimas_mejores < len(distancias_anteriores)*10
    

def busqueda_local(problem, N):
  Nodos = list(problem.get_nodes())
      
  solucion_referencia = crear_solucion(Nodos)
  
  distancia_final = 10000000
  solucion_final = solucion_referencia
  
  # Me guardo un vector de las distancias anteriores para poder decidir si 
  # estamos en un mínimo local, para poder escapar de él
  distancias_anteriores = []
  
  for i in range (N):
    solucion_vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(solucion_vecina, problem)
    
    if distancia_vecina < distancia_final:
      distancia_final = distancia_vecina
      solucion_final = solucion_vecina
      
      # Me guardo las distancias anteriores
      distancias_anteriores.append(distancia_final)
      # Si ya llevamos  unas cuantas distancias anteriores, vamos a comprobar
      if len(distancias_anteriores) >= 5:
        if deberiamos_parar(distancias_anteriores):
          print("Debo parar ya, me he ahorrado: " + str(N-i) + " iteraciones")
          break
        else:
          distancias_anteriores = []
          
    
    solucion_referencia = solucion_vecina
    
  print("Nuestra solución es: " + str(solucion_final))
  print("Nuestra distancia total es: " + str(distancia_final))
  return solucion_final
  
busqueda_local(problem, 100)

Debo parar ya, me he ahorrado: 71 iteraciones
Nuestra solución es: [0, 27, 2, 3, 4, 26, 18, 12, 11, 25, 29, 28, 6, 5, 13, 19, 20, 33, 34, 38, 22, 39, 21, 24, 40, 23, 41, 10, 8, 9, 30, 32, 35, 36, 31, 17, 37, 15, 16, 14, 7, 1]
Nuestra distancia total es: 1609


[0,
 27,
 2,
 3,
 4,
 26,
 18,
 12,
 11,
 25,
 29,
 28,
 6,
 5,
 13,
 19,
 20,
 33,
 34,
 38,
 22,
 39,
 21,
 24,
 40,
 23,
 41,
 10,
 8,
 9,
 30,
 32,
 35,
 36,
 31,
 17,
 37,
 15,
 16,
 14,
 7,
 1]

#### Recocido simulado

In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  
  vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  
  return vecina

# T = temperatura, d = distancia de soluciones
def probabilidad(T,d):
  r=random.random();
  # exp(-delta/Temperatura) siendo delta diferencia entre distancia valor actual y nueva
  if(r <= (e**(-1*d)/(T*1.0))):
    return True
  else:
    return False
  
def bajar_temperatura(T):
  return T-1

In [16]:
def recocido_simulado(problem, TEMPERATURA):
  #problem = datos del problema
  #T = Temperatura
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  while TEMPERATURA > 0:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
    # Si la nueva vecina es mejor se cambia y si es peor se cambia según una 
    # probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    TEMPERATURA = bajar_temperatura(TEMPERATURA)
  
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

sol = recocido_simulado(problem, 10000)

La mejor solución encontrada es [0, 6, 5, 13, 19, 37, 17, 31, 7, 1, 26, 18, 12, 11, 25, 41, 23, 40, 24, 8, 10, 4, 3, 2, 27, 29, 9, 21, 39, 22, 38, 30, 28, 32, 34, 33, 20, 35, 36, 15, 16, 14]
con una distancia total de 1658
